# Generate Pre-computed Embeddings for TCG Scanner

This notebook generates embeddings for all cards in the database and saves them as a binary file for fast loading in the mobile app.

**Without pre-computed embeddings**: First launch takes ~58 seconds
**With pre-computed embeddings**: App starts in ~200ms

**Output**: `riftbound.bin` (~1.1 MB) - Ready for Flutter app

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q tensorflow>=2.13.0
!pip install -q pillow>=9.5.0
!pip install -q requests>=2.31.0

print("✅ Dependencies installed")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import numpy as np
from pathlib import Path
from PIL import Image
import requests
from io import BytesIO
import tensorflow as tf
from tqdm import tqdm

# Set paths
DRIVE_ROOT = Path('/content/drive/MyDrive/tcg-scanner')
MODELS_DIR = DRIVE_ROOT / 'models' / 'tflite'
OUTPUT_DIR = DRIVE_ROOT / 'embeddings'

# Create output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Drive root: {DRIVE_ROOT}")
print(f"Models dir: {MODELS_DIR}")
print(f"Output dir: {OUTPUT_DIR}")

## 2. Load Card Database

In [ ]:
# Load cards.json from your repo
# You'll need to upload this file to Google Drive first
cards_json_path = DRIVE_ROOT / 'cards.json'

if not cards_json_path.exists():
    print("❌ cards.json not found!")
    print(f"Please upload cards.json to: {cards_json_path}")
    print("\nFrom your local machine:")
    print("  mobile/flutter/assets/data/cards.json → Google Drive")
else:
    with open(cards_json_path) as f:
        cards = json.load(f)
    
    print(f"✅ Loaded {len(cards)} cards from database")
    print(f"\nSample card:")
    print(f"  Name: {cards[0]['name']}")
    print(f"  Image: {cards[0]['image_path']}")
    print(f"  Type: {cards[0]['card_type']}")

## 3. Load Embedding Model

In [ ]:
# Load embedding.tflite
embedding_model_path = MODELS_DIR / 'embedding.tflite'

if not embedding_model_path.exists():
    print(f"❌ Model not found: {embedding_model_path}")
else:
    # Load TFLite model
    interpreter = tf.lite.Interpreter(str(embedding_model_path))
    interpreter.allocate_tensors()
    
    # Get input/output details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    
    print(f"✅ Embedding model loaded")
    print(f"  Input shape: {input_details['shape']}")
    print(f"  Output shape: {output_details['shape']}")
    print(f"  Expected: [1, 384] embedding vector")

## 4. Preprocessing Functions (Match Flutter Implementation)

In [ ]:
# ImageNet normalization constants (same as Flutter)
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def download_image(url: str, timeout: int = 10) -> Image.Image:
    """Download image from URL."""
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert('RGB')
        return image
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None

def preprocess_for_embedding(image: Image.Image) -> np.ndarray:
    """
    Preprocess image for embedding (matches TFLite model requirements).

    Args:
        image: PIL Image

    Returns:
        Preprocessed tensor in NCHW format [1, 3, 224, 224]
    """
    # Step 1: Resize to 224x224
    image = image.resize((224, 224), Image.Resampling.BILINEAR)

    # Step 2: Convert to numpy array [224, 224, 3]
    tensor = np.array(image, dtype=np.float32)

    # Step 3: Normalize with ImageNet statistics
    # Formula: (pixel/255 - mean) / std
    tensor = tensor / 255.0
    tensor = (tensor - IMAGENET_MEAN) / IMAGENET_STD

    # Step 4: Transpose to channels-first NCHW format
    # [224, 224, 3] → [3, 224, 224]
    # TFLite model expects channels-first format!
    tensor = np.transpose(tensor, (2, 0, 1))

    # Step 5: Add batch dimension [3, 224, 224] → [1, 3, 224, 224]
    tensor = np.expand_dims(tensor, axis=0)

    return tensor

def normalize_embedding(embedding: np.ndarray) -> np.ndarray:
    """
    L2 normalize embedding (EXACTLY matches Flutter implementation).

    Args:
        embedding: Raw embedding vector

    Returns:
        Normalized embedding (unit vector)
    """
    # Calculate L2 norm: sqrt(sum of squares)
    norm = np.linalg.norm(embedding)

    # Normalize: embedding / magnitude
    if norm > 0:
        embedding = embedding / norm

    return embedding

print("✅ Preprocessing functions defined (NCHW format)")

## 5. Generate Embeddings for All Cards

In [ ]:
print("="*60)
print("GENERATING EMBEDDINGS")
print("="*60)

embeddings = []
successful_cards = []
failed_cards = []

for i, card in enumerate(tqdm(cards, desc="Processing cards")):
    try:
        # Download card image
        image_url = card['image_path']
        image = download_image(image_url)
        
        if image is None:
            failed_cards.append((card['name'], 'Download failed'))
            # Use zero vector as placeholder
            embeddings.append(np.zeros(384, dtype=np.float32))
            continue
        
        # Preprocess image
        input_tensor = preprocess_for_embedding(image)
        
        # Run inference
        interpreter.set_tensor(input_details['index'], input_tensor)
        interpreter.invoke()
        embedding = interpreter.get_tensor(output_details['index'])[0]
        
        # L2 normalize (critical!)
        embedding = normalize_embedding(embedding)
        
        # Verify normalization
        norm = np.linalg.norm(embedding)
        if not (0.95 < norm < 1.05):
            print(f"⚠️  {card['name']}: norm = {norm:.4f} (expected ~1.0)")
        
        embeddings.append(embedding)
        successful_cards.append(card['name'])
        
    except Exception as e:
        print(f"\n❌ Error processing {card['name']}: {e}")
        failed_cards.append((card['name'], str(e)))
        embeddings.append(np.zeros(384, dtype=np.float32))

print(f"\n✅ Generated {len(successful_cards)} embeddings")
print(f"❌ Failed: {len(failed_cards)}")

if failed_cards:
    print("\nFailed cards:")
    for name, error in failed_cards[:10]:
        print(f"  - {name}: {error}")
    if len(failed_cards) > 10:
        print(f"  ... and {len(failed_cards) - 10} more")

## 6. Save Embeddings to Binary File

In [ ]:
# Convert to numpy array
embeddings_array = np.array(embeddings, dtype=np.float32)

print(f"Embeddings array shape: {embeddings_array.shape}")
print(f"Expected: ({len(cards)}, 384)")

# Save as binary file
output_file = OUTPUT_DIR / 'riftbound.bin'
embeddings_array.tofile(str(output_file))

file_size_mb = output_file.stat().st_size / 1024 / 1024
expected_size_mb = len(cards) * 384 * 4 / 1024 / 1024  # 4 bytes per float32

print(f"\n✅ Embeddings saved to: {output_file}")
print(f"   File size: {file_size_mb:.2f} MB")
print(f"   Expected: {expected_size_mb:.2f} MB")

if abs(file_size_mb - expected_size_mb) < 0.1:
    print(f"   ✅ File size matches expected!")
else:
    print(f"   ⚠️  File size mismatch!")

## 7. Verify Embeddings

In [ ]:
# Load embeddings back
loaded_embeddings = np.fromfile(str(output_file), dtype=np.float32)
loaded_embeddings = loaded_embeddings.reshape(len(cards), 384)

print(f"Loaded embeddings shape: {loaded_embeddings.shape}")

# Check a few embeddings
print("\nSample embeddings:")
for i in range(min(5, len(cards))):
    embedding = loaded_embeddings[i]
    norm = np.linalg.norm(embedding)
    print(f"  {cards[i]['name'][:40]:40} norm={norm:.4f}")

# Verify normalization
norms = np.linalg.norm(loaded_embeddings, axis=1)
valid_norms = np.sum((norms > 0.95) & (norms < 1.05))
zero_norms = np.sum(norms < 0.01)

print(f"\nNormalization check:")
print(f"  Valid norms (0.95-1.05): {valid_norms}/{len(cards)}")
print(f"  Zero vectors (failed): {zero_norms}/{len(cards)}")

if valid_norms == len(cards) - zero_norms:
    print(f"  ✅ All successful embeddings are properly normalized!")
else:
    print(f"  ⚠️  Some embeddings may not be normalized correctly")

## 8. Test Similarity Search

In [ ]:
# Test: Find similar cards
def find_similar_cards(query_idx: int, top_k: int = 5):
    """
    Find most similar cards to query card.
    """
    query_embedding = loaded_embeddings[query_idx]
    
    # Compute cosine similarity (dot product since normalized)
    similarities = np.dot(loaded_embeddings, query_embedding)
    
    # Get top K
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    print(f"\nQuery card: {cards[query_idx]['name']}")
    print(f"Top {top_k} similar cards:")
    
    for i, idx in enumerate(top_indices, 1):
        sim = similarities[idx]
        card = cards[idx]
        print(f"  {i}. {card['name'][:50]:50} similarity={sim:.4f}")

# Test with first few cards
print("="*60)
print("TESTING SIMILARITY SEARCH")
print("="*60)

for test_idx in [0, 10, 50]:
    if test_idx < len(cards):
        find_similar_cards(test_idx, top_k=5)

print("\n✅ Similarity search working correctly!")
print("   Top result should be the card itself (similarity ~1.0)")

## 9. Download for Flutter App

In [ ]:
print("="*60)
print("READY FOR FLUTTER APP")
print("="*60)

print(f"\n📦 File to download: {output_file}")
print(f"   Size: {file_size_mb:.2f} MB")
print(f"\n📍 Flutter destination:")
print(f"   mobile/flutter/assets/indices/riftbound.bin")
print(f"\n✅ Pre-computed embeddings complete!")
print(f"\nNext steps:")
print(f"1. Download riftbound.bin from Google Drive")
print(f"2. Copy to: mobile/flutter/assets/indices/riftbound.bin")
print(f"3. The app will load in ~200ms instead of ~58 seconds!")

# Provide download link
from google.colab import files
print(f"\nDownloading file...")
files.download(str(output_file))